In [1]:
import json
from datetime import datetime


filename = './corona-out-2'
# filename = './corona-out-3'

In [2]:
def to_timestamp(val):
    dt = val.split(' ')
    st = f"{dt[5]}-{dt[1]}-{dt[2]} {dt[3]}"
    dt = datetime.strptime(st, "%Y-%b-%d %H:%M:%S")
    return int(dt.timestamp())

def to_dtstring(val):
    return datetime.fromtimestamp(val).strftime('%b %d, %Y - %X')

def get_hashtags(data):
    hashtags = []
    for item in data:
        hashtags.append(item['text'])
    return hashtags

In [3]:
# def add_user(user):
#     if (user['id'] not in users):
#         users[user['id']] = user

# def add_tweet(tweet, tweet_type):
#     if tweet_type=='tweet':
#         if tweet['id'] not in tweets:
#             tweets[tweet['id']] = tweet
#     elif tweet_type=='retweet':
#         if tweet['id'] not in retweets:
#             retweets[tweet['id']] = tweet

# def add_retweet(retweet):
#     if (retweet['id'] not in retweets):
#         retweets[retweet['id']] = retweet

def add_user(user):
    if (user['id'] not in users):
        new_user = {}
        
        new_user['id'] = user['id']
        new_user['name'] = user['name']
        new_user['screen_name'] = user['screen_name']
        new_user['location'] = user['location']
        new_user['url'] = user['url']
        new_user['description'] = user['description']
        new_user['verified'] = user['verified']
        new_user['followers_count'] = user['followers_count']
        new_user['friends_count'] = user['friends_count']
        new_user['listed_count'] = user['listed_count']
        new_user['favourites_count'] = user['favourites_count']
        new_user['statuses_count'] = user['statuses_count']
        new_user['created_at'] = to_timestamp(user['created_at'])
        
        users[user['id']] = new_user


def process_tweet(tweet):
    new_tweet = {}
        
    new_tweet['created_at'] = to_timestamp(tweet['created_at'])
    new_tweet['id'] = tweet['id']
    if tweet['truncated']:
        new_tweet['text'] = tweet['extended_tweet']['full_text']
        new_tweet['hashtags'] = get_hashtags(tweet['extended_tweet']['entities']['hashtags'])
    else:
        new_tweet['text'] = tweet['text']
        new_tweet['hashtags'] = get_hashtags(tweet['entities']['hashtags'])

    user = {}
    user['id'] = tweet['user']['id']
    user['name'] = tweet['user']['name']
    user['screen_name'] = tweet['user']['screen_name']
    user['verified'] = tweet['user']['verified']
#     modified
    user['friends_count'] = tweet['user']['friends_count']
    user['followers_count'] = tweet['user']['followers_count']
    user['description'] = tweet['user']['description']
    user['created_at'] = tweet['user']['created_at']

    new_tweet['user'] = user

    new_tweet['source'] = tweet['source']
    new_tweet['quote_count'] = tweet['quote_count']
    new_tweet['reply_count'] = tweet['reply_count']
    new_tweet['retweet_count'] = tweet['retweet_count']
    new_tweet['favorite_count'] = tweet['favorite_count']
    
    if 'retweeted_status' in tweet:
        new_tweet['retweeted_status'] = process_tweet(tweet['retweeted_status'])
        
    new_tweet['is_quote_status'] = tweet['is_quote_status']
    if tweet['is_quote_status'] and 'quoted_status' in tweet:
        new_tweet['quoted_status'] = process_tweet(tweet['quoted_status'])
    
    return new_tweet
        
def add_tweet(tweet, tweet_type):
    if (tweet['id'] not in tweets):
        
        new_tweet = process_tweet(tweet)
        
        if tweet_type=='tweet':
            tweets[tweet['id']] = new_tweet
        elif tweet_type=='retweet':
            retweets[tweet['id']] = new_tweet
        elif tweet_type=='quote':
            quotes[tweet['id']] = new_tweet

In [4]:
def process_data(data):
    if data['id'] in tweets:
        return
    
    add_user(data['user'])

    if data['text'].startswith('RT'):
        add_tweet(data, 'retweet')
        if 'retweeted_status' in data:
            process_data(data['retweeted_status'])

    else:
        add_tweet(data, 'tweet')

In [5]:
users = {}
tweets = {}
retweets = {}
quotes = {}

for line in open(filename, 'r'):
    if len(line.strip()) == 0:
        continue
        
#     try:
    data = json.loads(line)
    process_data(data)

#     except:
#         continue

In [ ]:
# (7349, 11157, 15505)
# (10987, 11157, 18461)

len(tweets), len(retweets), len(users), len(quotes)

In [ ]:
idxs = list(retweets.keys())

i = 0
for idx in idxs:
    if i==10: break
    print(retweets[idx]['text'])
    print(retweets[idx]['retweeted_status']['text'])
    print()
    i+=1

In [ ]:
retweets[idxs[1]]

In [ ]:
retweets[idxs[0]].keys()

In [ ]:
idxs = list(tweets.keys())

ls = []

i = 0
for idx in idxs:
    if i==10: break
    print(tweets[idx])
    ls.append(tweets[idx])
    print()
    i+=1

In [ ]:
print(ls)

In [ ]:
tweets[idxs[0]]

In [ ]:
tweets[idxs[1]].keys()

In [ ]:
ids = list(users.keys())

users[ids[0]]